
# Sample Bin Packing Fitness

This notebook is to figure out ways to represent the problem landscape, and write fitness functions appropriate for the representation.


[Offline bin-packing](https://en.wikipedia.org/wiki/Bin_packing_problem#Offline_algorithms) - Ability to see all items before placing them into bins and allows improved approximation ratios.


## Things to figure out

- [ ] Representation of problem landscape
- [ ] Fitness function based on representation
- [ ] Determine how could selection, crossover, and mutation be edited to fit new representation
- [ ] Read file and store data in appropriate formats



# Reading Problem File


In [8]:
# open binpacking.txt

with open("Binpacking.txt", "r") as file:
    lines = file.readlines()


for _ in range(17):
    # removes till 'BPP      1'
    lines.pop(0)

# from here, text file repeats in pattern
# BPP      i
# number of different weights
# capacity of bin
# weight          number of items (repeats number of different weights times)

num_weights = []
capacities = []
item_weights = []

# loop through 5 BPP scenarios
for i in range(5):
    # remove seperator BPP i
    lines.pop(0)
    
    # saves number of different weights
    # and capacity of bin
    num_weights.append(int(lines.pop(0)))
    capacities.append(int(lines.pop(0)))
    
    # make new map for BPP scenario i
    item_weights.append({})
    # loop through number of different weights
    for j in range (num_weights[i]):
        # split line into weight and number of items with weight
        line = lines.pop(0).split()
        # map weight to number of items with weight
        item_weights[i][int(line[0])] = int(line[1])

# close file, idk if this is needed
file.close()

# debug print for sanity check
# cuz im going insane
print(item_weights)
print(capacities)
print(num_weights)

[{200: 3, 199: 1, 198: 2, 197: 2, 194: 2, 193: 1, 192: 1, 191: 3, 190: 2, 189: 1, 188: 2, 187: 2, 186: 1, 185: 4, 184: 3, 183: 3, 182: 3, 181: 2, 180: 1, 179: 4, 178: 1, 177: 4, 175: 1, 174: 1, 173: 2, 172: 1, 171: 3, 170: 2, 169: 3, 167: 2, 165: 2, 164: 1, 163: 4, 162: 1, 161: 1, 160: 2, 159: 1, 158: 3, 157: 1, 156: 6, 155: 3, 154: 2, 153: 1, 152: 3, 151: 2, 150: 4}, {200: 2, 199: 4, 198: 1, 197: 1, 196: 2, 195: 2, 194: 2, 193: 1, 191: 2, 190: 1, 189: 2, 188: 1, 187: 2, 186: 1, 185: 2, 184: 5, 183: 1, 182: 1, 181: 3, 180: 2, 179: 2, 178: 1, 176: 1, 175: 2, 174: 5, 173: 1, 172: 3, 171: 1, 170: 4, 169: 2, 168: 1, 167: 5, 165: 2, 164: 2, 163: 3, 162: 2, 160: 2, 159: 2, 158: 2, 157: 4, 156: 3, 155: 2, 154: 1, 153: 3, 152: 2, 151: 2, 150: 2}, {200: 1, 199: 2, 197: 2, 196: 2, 193: 3, 192: 2, 191: 2, 190: 2, 189: 3, 188: 1, 187: 1, 185: 3, 183: 2, 182: 1, 181: 3, 180: 3, 179: 3, 178: 1, 177: 5, 176: 2, 175: 5, 174: 4, 173: 1, 171: 3, 170: 1, 169: 2, 168: 5, 167: 1, 166: 4, 165: 2, 163: 1, 16

# Code


## Sample Bin-packing scenario

3 items.


In [ ]:
NUM_ITEMS = 3
ITEM_WEIGHTS = [2, 2, 3]
MAX_CAPACITY = 3
BINS = []

# have a dict of items and their weights?
items = {'item1' : 2, 'item2' : 2, 'item3' : 3}

In [ ]:
# fitness function 
# less bins = good

def fitness(BINS):
    if BINS == 1:
        score = 0
    else:
        score = len(BINS)

# another idea for fitness function
# invovles ratio of weights in bin to capacity of bin
# as well as number of bins

In [ ]:
# randomly allocate items to bins
def init(NUM_ITEMS, ITEM_WEIGHT, MAX_CAPACITY):

    for item in NUM_ITEMS:
        